In [1]:
import numpy as np
import pandas as pd

In [2]:
from lightautoml_gpu.reader.gpu.cudf_reader import CudfReader
from lightautoml_gpu.reader.base import PandasToPandasReader

from lightautoml_gpu.transformers.base import SequentialTransformer

from lightautoml_gpu.pipelines.utils import get_columns_by_role

from lightautoml_gpu.transformers.gpu import numeric_gpu, categorical_gpu, datetime_gpu
from lightautoml_gpu.transformers import numeric, categorical, datetime

from lightautoml_gpu.tasks import Task
from lightautoml_gpu.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML

import pandas as pd
import time
from sklearn.model_selection import train_test_split

from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import cudf

Level "INFO2: 17" already defined, skipping...
Level "INFO3: 13" already defined, skipping...


In [3]:
features = pd.read_csv('../../data/small_new/multilabel/train Data.csv')
labels = pd.read_csv('../../data/small_new/multilabel/train labels.csv')
labels.drop(columns='id', inplace=True)
data = pd.concat([features, labels], axis=1)
data = data[['n_0000','n_0001','n_0002','n_0003',
             'n_0004','n_0005','n_0006','n_0007',
             'service_a', 'service_b', 'service_c', 
             'service_d', 'service_e', 'service_f',
             'service_g', 'service_h', 'service_i',
             'service_j', 'service_k', 'service_l',
             'service_m', 'service_n']]

tr_data, te_data = train_test_split(
    data, 
    test_size=0.2,  
    random_state=42
)

roles = {
    "target": {'service_a', 'service_b', 'service_c',
               'service_d', 'service_e', 'service_f',
               'service_g', 'service_h', 'service_i',
               'service_j', 'service_k', 'service_l',
               'service_m', 'service_n'},
    "drop" : ['id']
}

/tmp/ipykernel_8024/705777730.py:1: DtypeWarning: Columns (329,331,333,336,338,344,345,346,348,354,355,356,357,358,361,362,364,367,372,377,380,383,385,387,390,392,399,400,406,408,409,413,416,418,419,431,433,437,438,442,448,449,450,453,457,464,473,478,479,481,483,485,486,489,492,495,496,497,498,499,500,503,507,508,510,511,514,515,517,519,520,521,522,523,524,526,527,530,534,537,538,539,541,544,547,548,549,550,551,557,558,560,564,569,572,573,576,577,579,583,596,597,598,600,601,602,604,605,606,608,609,613,619,620,625,627,628,629,631,632,633,636,641,642,643,645,646,647,648,651,655,661,662,665,668,675,676,679,682,685,687,689,690,691,694,698,701,702,703,706,711,712,713,719,720,721,733,735,737,738,742,746,747,748,749,752,754,755,760,764,768,770,781,782,789,797,807,812,814,817,818,822,823,824,825,832,840,843,844,845,850,853,857,858,861,867,868,873,874,876,877,879,880,881,883,886,890,893,897,899,900,901,902,904,905,908,909,910,912,913,914,915,916,922,923,931,933,935,937,939,942,943,946,951,955,9

In [4]:
adv_roles = True

## Imports (for potential use)

In [5]:
# Imports from our package
from lightautoml_gpu.automl.base import AutoML

from lightautoml_gpu.automl.presets.gpu.tabular_gpu_presets import TabularAutoMLGPU, TabularUtilizedAutoMLGPU
from lightautoml_gpu.tasks import Task

from lightautoml_gpu.pipelines.features.gpu.lgb_pipeline_gpu import LGBSimpleFeaturesGPU, LGBAdvancedPipelineGPU
from lightautoml_gpu.pipelines.features.gpu.linear_pipeline_gpu import LinearFeaturesGPU

from lightautoml_gpu.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml_gpu.pipelines.features.linear_pipeline import LinearFeatures


from lightautoml_gpu.ml_algo.gpu.boost_cb_gpu import BoostCBGPU
from lightautoml_gpu.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml_gpu.ml_algo.gpu.linear_gpu import LinearLBFGSGPU

from lightautoml_gpu.ml_algo.boost_cb import BoostCB
from lightautoml_gpu.ml_algo.linear_sklearn import LinearLBFGS


from lightautoml_gpu.pipelines.ml.base import MLPipeline
from lightautoml_gpu.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector

## TabularAutoML

In [6]:
task = Task('multilabel')

multilabel isn`t supported in lgb


In [7]:
automl = TabularAutoML(
    task = task, 
    timeout = 3600,
    cpu_limit = 4,
    reader_params = {'n_jobs': 4, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'cb']]}
)

In [8]:
oof_pred = automl.fit_predict(data, roles = roles, verbose = 2)

[12:13:50] Stdout logging level is INFO2.
[12:13:50] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[12:13:50] Task: multilabel

[12:13:50] Start automl preset with listed constraints:
[12:13:50] - time: 3600.00 seconds
[12:13:50] - CPU: 4 cores
[12:13:50] - memory: 16 GB

[12:13:50] Train data shape: (14644, 22)



/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[12:14:00] Feats was rejected during automatic roles guess: []
[12:14:00] Layer 1 train process start. Time left 3590.21 secs
[12:14:00] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[12:14:00] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[12:14:03] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[12:14:05] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[12:14:08] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.350185599077338
[12:14:08] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[12:14:08] Time left 3582.07 secs



...

[12:14:25] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[12:14:36] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[12:14:46] Fitting Lvl_0_Pipe_1_Mod_0_CatBoost finished. score = -7.319639586394551
[12:14:46] Lvl_0_Pipe_1_Mod_0_CatBoost fitting and predicting completed
[12:14:46] Time left 3543.60 secs

[12:14:46] Layer 1 training completed.

[12:14:46] Blending: optimization starts with equal weights and score -7.3139995871566095
[12:14:46] Blending: iteration 0: score = -7.313940630217811, weights = [0.5436461  0.45635393]
[12:14:47] Blending: iteration 1: score = -7.313940630217811, weights = [0.5436461  0.45635393]
[12:14:47] Blending: no score update. Terminated

[12:14:47] Automl preset training completed in 56.78 seconds

[12:14:47] Model description:
Final prediction for new objects (level 0) = 
	 0.54365 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.45635 * (3 averaged models Lvl_0_Pipe_1_Mod_0_CatBoost) 



In [9]:
task = Task('multilabel', device='gpu')

multilabel isn`t supported in lgb
[12:14:49] CatBoost uses as obj. MultiCrossEntropy.


In [10]:
automl_gpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'pb']]}
)

In [11]:
oof_pred_gpu = automl_gpu.fit_predict(data, roles = roles, verbose = 2)

[12:14:49] Stdout logging level is INFO2.
[12:14:49] Task: multilabel

[12:14:49] Start automl preset with listed constraints:
[12:14:49] - time: 3600.00 seconds
[12:14:49] - CPU: 1 cores
[12:14:49] - memory: 16 GB

[12:14:49] Train data shape: (14644, 22)
[12:14:50] Feats was rejected during automatic roles guess: []
[12:14:50] Layer 1 train process start. Time left 3599.37 secs
[12:14:50] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[12:14:50] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[12:14:55] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[12:14:58] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[12:15:02] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.375488443139862
[12:15:02] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[12:15:02] Time left 3587.11 secs

[12:15:04] Stdout logging level is INFO.
[12:15:04] GDBT train starts. Max iter 3000, early stopping roun

In [12]:
gpu_inf = automl_gpu.predict(data)

In [13]:
automl_gpu.to_cpu()

multilabel isn`t supported in lgb
[12:15:14] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[12:15:14] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[12:15:14] CatBoost uses as obj. MultiCrossEntropy.


  0%|          | 0/224 [00:00<?, ?it/s]

multilabel isn`t supported in lgb
[12:15:15] CatBoost uses as obj. MultiCrossEntropy.


  0%|          | 0/139 [00:00<?, ?it/s]

  0%|          | 0/148 [00:00<?, ?it/s]

  0%|          | 0/138 [00:00<?, ?it/s]

multilabel isn`t supported in lgb
[12:15:16] CatBoost uses as obj. MultiCrossEntropy.


In [ ]:
automl_gpu.levels[0][1].ml_algos[0].name

In [ ]:
#model_inf.levels[0][1].ml_algos[0].name

In [14]:
cpu_inf = automl_gpu.predict(data)

In [15]:
from joblib import dump, load
import time
pickle_file = './gpu.joblib'

In [16]:
start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_gpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)

Raw dump duration: 0.015s


In [17]:

start = time.time()
with open(pickle_file, 'rb') as f:
    model_inf = load(f)
raw_load_duration = time.time() - start
print("Raw load duration: %0.3fs" % raw_load_duration)


Raw load duration: 0.009s


In [ ]:
model_inf.levels[0][1].ml_algos[0].models[0].__dict__

In [18]:
disk_pred = model_inf.predict(data)

In [19]:
disk_pred.data.T

array([[0.00409981, 0.01182499, 0.00467859, ..., 0.00900183, 0.0052202 ,
        0.00467147],
       [0.00137125, 0.00218978, 0.00143265, ..., 0.00498717, 0.00159189,
        0.00160492],
       [0.03862619, 0.07474752, 0.04614756, ..., 0.23067841, 0.03688202,
        0.04131494],
       ...,
       [0.01116886, 0.01822781, 0.01073365, ..., 0.0433928 , 0.0087622 ,
        0.01240759],
       [0.10287236, 0.16949932, 0.11420202, ..., 0.2631097 , 0.10364739,
        0.1352504 ],
       [0.628377  , 0.5847929 , 0.6263023 , ..., 0.3624467 , 0.63298523,
        0.59682274]], dtype=float32)

In [20]:
disk_pred.data.T

array([[0.00409981, 0.01182499, 0.00467859, ..., 0.00900183, 0.0052202 ,
        0.00467147],
       [0.00137125, 0.00218978, 0.00143265, ..., 0.00498717, 0.00159189,
        0.00160492],
       [0.03862619, 0.07474752, 0.04614756, ..., 0.23067841, 0.03688202,
        0.04131494],
       ...,
       [0.01116886, 0.01822781, 0.01073365, ..., 0.0433928 , 0.0087622 ,
        0.01240759],
       [0.10287236, 0.16949932, 0.11420202, ..., 0.2631097 , 0.10364739,
        0.1352504 ],
       [0.628377  , 0.5847929 , 0.6263023 , ..., 0.3624467 , 0.63298523,
        0.59682274]], dtype=float32)

In [21]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
print(oof_pred.data.T)
print()
print(disk_pred.data.T)

[[0.00409981 0.01182499 0.00467859 ... 0.00900183 0.0052202  0.00467147]
 [0.00137125 0.00218978 0.00143265 ... 0.00498717 0.00159189 0.00160492]
 [0.03862619 0.07474752 0.04614756 ... 0.23067841 0.03688202 0.04131494]
 ...
 [0.01116886 0.01822781 0.01073365 ... 0.0433928  0.0087622  0.01240759]
 [0.10287236 0.16949932 0.11420202 ... 0.2631097  0.10364739 0.1352504 ]
 [0.628377   0.5847929  0.6263023  ... 0.3624467  0.63298523 0.59682274]]

[[0.00409981 0.01182498 0.00467859 ... 0.00900183 0.0052202  0.00467147]
 [0.00137125 0.00218979 0.00143265 ... 0.00498716 0.00159189 0.00160492]
 [0.03862621 0.07474754 0.04614757 ... 0.23067841 0.03688202 0.04131496]
 ...
 [0.01116887 0.01822781 0.01073365 ... 0.0433928  0.0087622  0.0124076 ]
 [0.10287233 0.16949931 0.114202   ... 0.26310968 0.10364738 0.13525039]
 [0.628377   0.584793   0.62630224 ... 0.36244667 0.63298523 0.59682274]]

[[0.00396225 0.01328214 0.00533394 ... 0.00686121 0.00422175 0.0050306 ]
 [0.00132121 0.00203942 0.00131857 ..

In [ ]:
cluster = LocalCUDACluster(rmm_managed_memory=True, CUDA_VISIBLE_DEVICES="0",
                               protocol="ucx", enable_nvlink=True,
                               memory_limit="8GB")
print("dashboard:", cluster.dashboard_link)
client = Client(cluster)
client.run(cudf.set_allocator, "managed")

In [ ]:
task = Task('multilabel', device='mgpu')

automl_mgpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42, 'npartitions': 2},
    general_params = {'use_algos': [['xgb', 'linear_l2']]},
    client = client
)

oof_pred_mgpu = automl_mgpu.fit_predict(data, roles = roles, verbose = 2)

In [ ]:
automl_mgpu.to_cpu()

In [ ]:
mcpu_inf = automl_mgpu.predict(data)

In [ ]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
print(oof_pred.data.T)
print()
print(oof_pred_mgpu.data.T)
print()
print(mcpu_inf.data.T)

In [ ]:
from joblib import dump, load
import time

pickle_file = './mgpu.joblib'

start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_mgpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)